In [1]:
import pandas as pd
import os


# IMPORT DATA

In [2]:
#Move to correct directory
os.chdir("/Users/karina/Library/CloudStorage/Dropbox/Programming/OTI_Interview")

# Read the CSV file into a DataFrame
inspectionsDfRaw = pd.read_csv('DOHMH_New_York_City_Restaurant_Inspection_Results.csv')

#Summarize data
print(inspectionsDfRaw.columns)
inspectionsDfRaw.head()

#Keep just columns of interest
columnsOfInterest = ["CAMIS","BORO","INSPECTION DATE","ACTION","VIOLATION CODE","VIOLATION DESCRIPTION","INSPECTION TYPE","Community Board",]
inspDf = inspectionsDfRaw[columnsOfInterest]


Index(['CAMIS', 'DBA', 'BORO', 'BUILDING', 'STREET', 'ZIPCODE', 'PHONE',
       'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'VIOLATION CODE',
       'VIOLATION DESCRIPTION', 'CRITICAL FLAG', 'SCORE', 'GRADE',
       'GRADE DATE', 'RECORD DATE', 'INSPECTION TYPE', 'Latitude', 'Longitude',
       'Community Board', 'Council District', 'Census Tract', 'BIN', 'BBL',
       'NTA', 'Location Point1'],
      dtype='object')


# CLEAN AND FILTER ROWS

In [3]:
#Eliminate known irrelevant rows 

#(1) Inspections with date 1900 -- these are inspections not yet completed
inspDf = inspDf[inspDf["INSPECTION DATE"] != "01/01/1900"]

#(2) Actions that don't specifically detail whether violations were found
actions_to_exclude = ['Establishment re-opened by DOHMH.', 'Establishment re-closed by DOHMH.']
inspDf = inspDf[~inspDf["ACTION"].isin(actions_to_exclude)]


#Summarize current dataframe
print(inspDf.info())

<class 'pandas.core.frame.DataFrame'>
Index: 192222 entries, 7 to 198363
Data columns (total 8 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  192222 non-null  int64  
 1   BORO                   192222 non-null  object 
 2   INSPECTION DATE        192222 non-null  object 
 3   ACTION                 192222 non-null  object 
 4   VIOLATION CODE         191224 non-null  object 
 5   VIOLATION DESCRIPTION  191224 non-null  object 
 6   INSPECTION TYPE        192222 non-null  object 
 7   Community Board        189105 non-null  float64
dtypes: float64(1), int64(1), object(6)
memory usage: 13.2+ MB
None


In [4]:
#Eliminate rows with null community board values

#(Didn't spend time to figure out why these might occur, but < 2% of data)
rows_before = inspDf.shape[0]
inspDf = inspDf.dropna(subset=["Community Board"])
rows_after = inspDf.shape[0]
#Make community boards column integers
inspDf["Community Board"] = inspDf["Community Board"].astype(int)

#Summarize results
print(f"rows dropped = {rows_before - rows_after}")

rows dropped = 3117


In [5]:
#Keep only rows from 2022

inspDf["INSPECTION DATE"] = pd.to_datetime(inspDf["INSPECTION DATE"])
inspDf = inspDf[inspDf["INSPECTION DATE"].dt.year == 2022]


In [6]:
#Summarize data frame
print(inspDf.info())
print("\n** NOTE: Null VIOLATION CODE and DESCRIPTION rows are inspections with no violations found")

<class 'pandas.core.frame.DataFrame'>
Index: 88195 entries, 8 to 198363
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   CAMIS                  88195 non-null  int64         
 1   BORO                   88195 non-null  object        
 2   INSPECTION DATE        88195 non-null  datetime64[ns]
 3   ACTION                 88195 non-null  object        
 4   VIOLATION CODE         87903 non-null  object        
 5   VIOLATION DESCRIPTION  87903 non-null  object        
 6   INSPECTION TYPE        88195 non-null  object        
 7   Community Board        88195 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(5)
memory usage: 6.1+ MB
None

** NOTE: Null VIOLATION CODE and DESCRIPTION rows are inspections with no violations found


# DETERMINE WHAT TO COUNT

In [7]:
#This was to check if all restos have same number of violations, which they don't,
#so decided to count restos rather than inspections

grouped = inspDf.groupby("CAMIS").agg({'VIOLATION CODE': 'nunique'})

#Explain Conclusion:
print(grouped.describe())
print('''\n
      Since restaurants have different numbers of inspections (mean = 4, stdev = 2.8),
      I am going to look at number of restaurants with violations rather than absolute
      number of violations. I looked briefly at some example restaurants with multiple
      inspections using this line of code:
      
      inspDf[inspDf["CAMIS"].isin([30112340,40356483])].sort_values("CAMIS")
      
      and didn't see an obvious way to make sense of what determined how often a resto
      was inspected or how many separate inspections were recorded per inspection date
      for a given restaurant.
      ''')

       VIOLATION CODE
count    19438.000000
mean         4.085966
std          2.760411
min          0.000000
25%          2.000000
50%          3.000000
75%          6.000000
max         21.000000


      Since restaurants have different numbers of inspections (mean = 4, stdev = 2.8),
      I am going to look at number of restaurants with violations rather than absolute
      number of violations. I looked briefly at some example restaurants with multiple
      inspections using this line of code:
      
      inspDf[inspDf["CAMIS"].isin([30112340,40356483])].sort_values("CAMIS")
      
      and didn't see an obvious way to make sense of what determined how often a resto
      was inspected or how many separate inspections were recorded per inspection date
      for a given restaurant.
      


# EXTRACT SUMMARY DATA

In [8]:
#Separate rodent-related violations from all other inspection results

rodentViolationCodes = ['04K', '04L', '08A'];

#Replace all inspections without rodent violations with 0
inspDf.loc[~inspDf["VIOLATION CODE"].isin(rodentViolationCodes), rodentViolationCodes] = 0

#Make a dataframe of just inspections with rodent-related violations
rodentViolationsDf = inspDf.copy()
rodentViolationsDf = rodentViolationsDf[rodentViolationsDf["VIOLATION CODE"].isin(rodentViolationCodes)]

In [9]:
#Create dataframe with per community board final counts of inspected restos and restos with rodent violations

#Count unique restaurants with and without rodent-related violations
uniqueRestaurantsInspected = inspDf.groupby("Community Board").agg(inspected_restos = ('CAMIS','nunique'))
uniqueRestaurantsWithRodentViolations = rodentViolationsDf.groupby("Community Board").agg(rodent_viol_restos = ('CAMIS','nunique'))

#combine total inspections and inspections with rodent violations in same table
rodentViolations = pd.merge(uniqueRestaurantsInspected, uniqueRestaurantsWithRodentViolations, on='Community Board', how='left')
rodentViolations = rodentViolations.fillna(0) #any CB's with no rodent violating restos, fill in 0 for that column
rodentViolations = rodentViolations.astype(int)

#Preview
rodentViolations.head()


,inspected_restos,rodent_viol_restos
Community Board,,
101,589,184
102,883,387
103,944,408
104,766,348
105,1676,555


In [10]:
#Save to CSV

rodentViolations.to_csv('rodentViolationSummary.csv', index=True) 